# Imports

In [1]:
import pandas as pd

# Data Collection

In [2]:
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Potion_Classification/train_X_poc.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Potion_Classification/train_Y_poc.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Potion_Classification/test_X_poc.csv

--2022-08-03 09:56:24--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Potion_Classification/train_X_poc.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)... 52.219.158.54
Connecting to nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)|52.219.158.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 312331 (305K) [text/csv]
Saving to: ‘train_X_poc.csv’

train_X_poc.csv     100%[===================>] 305.01K   350KB/s    in 0.9s    

2022-08-03 09:56:26 (350 KB/s) - ‘train_X_poc.csv’ saved [312331/312331]

--2022-08-03 09:56:26--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/CCBP_Platform_Projects/Potion_Classification/train_Y_poc.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com 

In [3]:
train_X_df = pd.read_csv('train_X_poc.csv')
train_Y_df = pd.read_csv('train_Y_poc.csv', header=None).squeeze()
test_X_df = pd.read_csv('test_X_poc.csv')

# Implementation

In [4]:
train_X_df.head()

,peppermint,dittany,lavender,ginger_root_width(mm),powdered_dragon_claw(mg),fluxweed(mg),drops_of_tree_wood_oil,fire_seed,rose_petal,daisy_root_width,daisy_root_height,water(ml),dragon_blood(ml),porcupine_quills,drops_of_red_wine,gillyweed(mg),honey(mg),morning_dew,ginger_root_height(mm),leech_juice(ml)
0,1.0,0.0,1.0,0.172230,0.10,0.857143,0.426243,0.0,0.0,0.722222,0.714286,0.656417,0.111111,0.000000,0.055556,56.0,4.1,1.0,0.272447,0.800000
1,1.0,1.0,1.0,0.338451,0.45,0.142857,0.200160,1.0,0.0,0.111111,0.500000,0.233957,1.000000,0.666667,0.388889,56.0,4.3,0.0,0.404310,0.175000
2,1.0,1.0,1.0,0.638184,0.60,1.000000,0.743453,0.0,1.0,0.388889,0.714286,0.635027,0.555556,0.888889,0.277778,59.0,2.5,0.0,0.416624,0.841667
3,1.0,0.0,0.0,0.830441,0.50,0.714286,0.194816,1.0,0.0,0.611111,0.500000,0.649064,0.444444,0.277778,0.055556,38.0,4.8,1.0,0.744997,0.416667
4,1.0,1.0,0.0,0.080774,0.80,0.714286,0.728220,1.0,1.0,0.333333,0.714286,0.307487,0.222222,0.277778,0.555556,53.0,3.5,1.0,0.284248,0.600000


In [5]:
train_X_df.isna().sum()

peppermint                  0
dittany                     0
lavender                    0
ginger_root_width(mm)       0
powdered_dragon_claw(mg)    0
fluxweed(mg)                0
drops_of_tree_wood_oil      0
fire_seed                   0
rose_petal                  0
daisy_root_width            0
daisy_root_height           0
water(ml)                   0
dragon_blood(ml)            0
porcupine_quills            0
drops_of_red_wine           0
gillyweed(mg)               0
honey(mg)                   0
morning_dew                 0
ginger_root_height(mm)      0
leech_juice(ml)             0
dtype: int64

In [6]:
train_Y_df

0       1
1       0
2       3
3       1
4       2
       ..
1120    3
1121    3
1122    0
1123    2
1124    1
Name: 0, Length: 1125, dtype: int64

In [8]:
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
pipe = Pipeline(steps = [('scaler', MinMaxScaler()),
                         ('classifier', LogisticRegression(solver='liblinear', max_iter=2000))])
param_distributions = [
              {
                'scaler': [MinMaxScaler(), RobustScaler(),StandardScaler(), 'passthrough'],
                'classifier': [LogisticRegression(solver='liblinear', max_iter=5000)],
                'classifier__penalty': ['l2', 'l1','elasticnet'],
                'classifier__C':[1,2,3,4,0.01,0.001,0.0001]
              },
              {
                'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
                'classifier': [KNeighborsClassifier()],
                'classifier__n_neighbors': range(1,30),
                'classifier__p': [1, 2, 3, 4,5,6]
              }
            ]
random_search_cv = RandomizedSearchCV(pipe, param_distributions=param_distributions, n_iter=30, scoring='f1_weighted', refit=True, cv=20, random_state=0) 
random_search_cv.fit(train_X_df, train_Y_df)
print(random_search_cv.best_params_)

{'scaler': 'passthrough', 'classifier__penalty': 'l1', 'classifier__C': 3, 'classifier': LogisticRegression(C=3, max_iter=5000, penalty='l1', solver='liblinear')}


In [9]:
random_search_cv.cv_results_['mean_test_score'].max()

0.8487677641277293

In [10]:
best_model = random_search_cv.best_estimator_
predicted_test_Y = best_model.predict(test_X_df)
predicted_test_Y

array([0, 0, 2, 0, 2, 0, 1, 0, 3, 0, 2, 2, 0, 0, 1, 3, 1, 0, 3, 1, 3, 3,
       3, 2, 1, 0, 3, 1, 0, 3, 1, 0, 3, 1, 2, 2, 0, 2, 1, 0, 0, 0, 1, 1,
       3, 0, 0, 2, 3, 2, 0, 1, 3, 1, 1, 3, 3, 3, 1, 1, 1, 0, 2, 2, 2, 2,
       3, 0, 3, 1, 0, 1, 3, 1, 3, 1, 2, 0, 2, 0, 0, 2, 1, 2, 3, 0, 0, 3,
       3, 0, 0, 3, 0, 0, 3, 2, 1, 3, 3, 2, 1, 0, 0, 2, 0, 0, 3, 3, 1, 0,
       3, 1, 0, 0, 1, 1, 1, 3, 3, 2, 1, 2, 1, 0, 0, 2, 3, 3, 2, 1, 1, 3,
       1, 2, 0, 1, 1, 1, 2, 3, 0, 0, 3, 1, 3, 2, 2, 3, 0, 0, 3, 0, 1, 1,
       0, 3, 1, 3, 3, 1, 1, 3, 0, 1, 3, 1, 0, 2, 0, 2, 2, 3, 1, 2, 0, 0,
       0, 1, 0, 3, 0, 2, 2, 0, 2, 0, 0, 2, 3, 0, 1, 1, 0, 3, 2, 2, 1, 3,
       1, 0, 3, 1, 2, 3, 1, 2, 2, 2, 3, 0, 3, 3, 3, 0, 3, 2, 3, 2, 3, 0,
       0, 1, 0, 1, 0, 0, 3, 0, 2, 2, 1, 2, 3, 1, 2, 2, 1, 0, 2, 2, 1, 0,
       0, 1, 1, 3, 0, 2, 2, 2, 0, 0, 1, 2, 2, 1, 3, 2, 0, 3, 1, 3, 1, 3,
       3, 3, 2, 2, 2, 1, 3, 0, 0, 3, 3, 2, 2, 1, 1, 2, 2, 1, 0, 0, 1, 3,
       1, 3, 2, 0, 2, 0, 2, 0, 0, 0, 0, 2, 1, 0, 0,

# Submission 
Writing the predicted values to CSV file

In [11]:
predicted_test_Y = pd.DataFrame(predicted_test_Y).to_csv('predicted_test_Y_poc.csv', header=None, index=False)
from google.colab import files
files.download('predicted_test_Y_poc.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>